In [18]:
import math
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
from tabulate import tabulate

import warnings
warnings.filterwarnings('ignore')

sns.reset_defaults()
sns.set_theme(rc={'figure.dpi': 72, 'savefig.dpi': 300,
              'figure.autolayout': True})
sns.set_style('ticks')
sns.set_context('paper')


In [19]:
# Wikipedia vote network dataset
# https://snap.stanford.edu/data/wiki-Vote.html
# Note: It is a directed graph, but we ignore the directions!

G = nx.read_edgelist('../datasets/wiki-Vote.txt.gz', nodetype=int)
print(nx.info(G))

Graph with 7115 nodes and 100762 edges


In [27]:
def modularity(G, S):
    m = sum(dict(G.degree).values()) / 2

    def f(s):
        # Number of intra-partition edges
        E_s = sum(1 for u, v in G.edges(s) if v in s)

        # Sum of degrees of the nodes in the partition
        K_s = sum(G.degree[u] for u in s)

        return E_s / m - (K_s / (2 * m)) ** 2

    Q = sum(map(f, S))
    return Q


def largest_eig(B, power_method=True, max_iter=100, tol=1e-6):
    if power_method:
        # Find the largest eigenvalue and eigenvector using the power method.
        n = B.shape[1]
        v = np.ones(n) * (1 / math.sqrt(n))
        lam_prev = 0
        for i in range(max_iter):
            v = B @ v
            v /= np.linalg.norm(v)
            lam = (v.T @ B @ v) / (v.T @ v)
            if np.abs(lam - lam_prev) < tol:
                break
            lam_prev = lam
        return lam, v
    else:
        # Find the largest eigenvalue and eigenvector using the NumPy ready-to-use function.
        lam, X = np.linalg.eig(B)
        lam, X = np.real_if_close(lam), np.real_if_close(X)
        i = np.argsort(lam)
        l, x = lam[i[-1]], X[:, i[-1]]
        return np.array(l, dtype=float), np.array(x, dtype=float)


def find_coms(G):
    """Community Detection Using Fast Modularity Optimization Algorithm"""
    coms = []  # Indivisible communities
    check = [set(G.nodes)]

    # Repeat hierarchically.
    # If all communities are indivisible, stop.
    while len(check) > 0:
        g = G.subgraph(check.pop(0))
        node = list(g)
        A = nx.to_scipy_sparse_array(g, dtype=int)  # Adjacency matrix

        # Calculate the modularity matrix
        k = A.sum(axis=1)
        m = k.sum() / 2
        K = np.outer(k, k) / (2 * m)  # Expected adjacency matrix
        B = np.array(np.asmatrix(A - K))  # Modularity matrix

        # Divide the nodes by the signs of the elements of x_n
        lam, x = largest_eig(B)
        c1 = {node[i] for i, v in enumerate(x) if v >= 0}
        c2 = set(g.nodes) - c1

        if len(c1) == 0 or len(c2) == 0:
            coms.append(set(g.nodes))
            continue

        Q_prev = modularity(G, [*coms, *check, set(g.nodes)])
        Q = modularity(G, [*coms, *check, c1, c2])

        if Q < Q_prev:
            # If a proposed split does not cause modularity to increase,
            # declare community indivisible and do not split it.
            coms.append(set(g.nodes))
        else:
            check.append(c1)
            check.append(c2)

    return coms


In [28]:
coms = find_coms(G)
print("Results for the Fast Modularity Optimization Algorithm:")
print("Number of detected communities:", len(coms))
print("Modularity:", round(modularity(G, coms), 4))


Results for the Fast Modularity Optimization Algorithm:
Number of detected communities: 3
Modularity: 0.3998


In [22]:
gcoms = nx.community.greedy_modularity_communities(G)
print("Results for the NetworkX Greedy Algorithm:")
print("Number of detected communities:", len(gcoms))
print("Modularity:", round(modularity(G, gcoms), 4))


Results for the NetworkX Greedy Algorithm:
Number of detected communities: 55
Modularity: 0.3402


In [23]:
lcoms = nx.community.louvain_communities(G, seed=2)
print("Results for the NetworkX Louvain Algorithm:")
print("Number of detected communities:", len(lcoms))
print("Modularity:", round(modularity(G, lcoms), 4))


Results for the NetworkX Louvain Algorithm:
Number of detected communities: 29
Modularity: 0.4264


In [24]:
# Save detected communities using
# Fast Modularity Optimization Algorithm to file.
with open('communities.txt', 'w') as f:
    for i, c in enumerate(coms, 1):
        f.write("#"*10 + f" Community {i}:\n\n")
        f.write(str(c) + '\n'*5)
